# This notebook trains a model with cross-val on the entire dataset
- and gets it's pred_probs and saves results as numpy files
- make sure we run entirety of evaluate benchmarks to get consensus labeling first

In [8]:
# %load_ext autoreload
# %autoreload 2

import sys

sys.path.insert(0, "../")

from autogluon.vision import ImagePredictor, ImageDataset
import numpy as np
import pandas as pd
import pickle
import datetime
from pathlib import Path
import cleanlab
from utils.cross_validation_autogluon import cross_val_predict_autogluon_image_dataset

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [9]:
!pwd

/Users/uly/Documents/Cleanlab/h_cleanlab/multiannotator_benchmarks


In [14]:
# Load test files
data_filepath = './data/model_data_070622/cifar10_test_consensus_dataset.csv'
df = pd.read_csv(data_filepath)

# Create mini train dataset
num_from_each_group = 15
mini_df = df.groupby("label").head(num_from_each_group)
mini_df.groupby("label")["image"].count().reset_index()

,label,image
0,0,15
1,1,15
2,2,15
3,3,15
4,4,15
5,5,15
6,6,15
7,7,15
8,8,15
9,9,15


In [ ]:
# # data path
# CIFAR_10_DATA_PATH = "/Data/cifar10_png/"

# # read data from root folder
# train_dataset, _, test_dataset = \
#     ImageDataset.from_folders(
#         root=CIFAR_10_DATA_PATH,
#     )

In [19]:
%%time

# generate cross-validated predicted probabilities for various models so we can use them for ensemble scoring methods
models = [
    "resnet18",
    "swin_base_patch4_window7_224"
]

epochs = 1
holdout_frac = 0.2
n_splits = 5

# run cross-validation for each model
for model in models:
    
    print("----")
    print(f"Running cross-validation for model: {model}")

    MODEL_PARAMS = {
        "model": model,
        "epochs": epochs,
        "holdout_frac": holdout_frac,
    }

    # results of cross-validation will be saved to pickle files for each model/fold
    _ = \
        cross_val_predict_autogluon_image_dataset(
            dataset=mini_df,
            out_folder=f"./data/cifar10_test_dataset_{model}/", # save results of cross-validation in pickle files for each fold
            n_splits=n_splits,
            model_params=MODEL_PARAMS,
            time_limit=600,
        )

----
Running cross-validation for model: resnet18
----
Running Cross-Validation on Split: 0


ValueError: Insufficient detected # gpus 0 vs requested 1

## Read pickle files from cross-validation

In [20]:
def load_pickle(pickle_file_name):
    """Load pickle file"""

    print(f"Loading {pickle_file_name}")

    with open(pickle_file_name, 'rb') as handle:
        out = pickle.load(handle)
        
    return out

In [ ]:
# test file read
for model in models:
    out_subfolder = f"./data/cifar10_test_dataset_{model}/split_0/"
    split_num = 0

    get_pickle_file_name = (
        lambda object_name: f"{out_subfolder}_{object_name}_split_{split_num}"
    )

    pred_probs = load_pickle(get_pickle_file_name("test_pred_probs"))
    pred_features = load_pickle(get_pickle_file_name("test_pred_features"))
    labels = load_pickle(get_pickle_file_name("test_labels"))
    images = load_pickle(get_pickle_file_name("test_image_files"))
    indices = load_pickle(get_pickle_file_name("test_indices"))